In [0]:
pip install mlflow==2.21.3


In [0]:
import pandas as pd
import numpy as np
import boto3
import io
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import tempfile
from sklearn.model_selection import train_test_split


s3 = boto3.client('s3')
bucket = "columbia-gr5069-main"

# Define a helper to read S3 CSV
def read_s3_csv(key):
    obj = s3.get_object(Bucket=bucket, Key=key)
    return pd.read_csv(io.BytesIO(obj['Body'].read()))

# Load datasets from S3
results = read_s3_csv("raw/results.csv")
races = read_s3_csv("raw/races.csv")
drivers = read_s3_csv("raw/drivers.csv")
lap_times = read_s3_csv("raw/lap_times.csv")
pit_stops = read_s3_csv("raw/pit_stops.csv")
qualifying = read_s3_csv("raw/qualifying.csv")


In [0]:
# -------------------
# JOIN & FEATURE CREATION SECTION
# -------------------
# Join: results + races + drivers
results_merged = results.merge(races, on="raceId", suffixes=("", "_race"))
results_merged = results_merged.merge(drivers, on="driverId", suffixes=("", "_driver"))

# Feature: average lap time per driver per race
lap_avg = lap_times.groupby(['raceId', 'driverId'])['milliseconds'].mean().reset_index()
lap_avg.rename(columns={'milliseconds': 'avg_lap_time_ms'}, inplace=True)

# Feature: number of pit stops per driver per race
pit_count = pit_stops.groupby(['raceId', 'driverId']).size().reset_index(name='num_pit_stops')

# Feature: qualifying position (lowest value if multiple attempts)
qualifying_agg = qualifying.groupby(['raceId', 'driverId'])['position'].min().reset_index()
qualifying_agg.rename(columns={'position': 'qualifying_position'}, inplace=True)

# Merge engineered features
results_merged = results_merged.merge(lap_avg, on=['raceId', 'driverId'], how='left')
results_merged = results_merged.merge(pit_count, on=['raceId', 'driverId'], how='left')
results_merged = results_merged.merge(qualifying_agg, on=['raceId', 'driverId'], how='left')

# Select features for modeling
model_data = results_merged[[
    'raceId', 'driverId', 'grid', 'positionOrder', 'points',
    'avg_lap_time_ms', 'num_pit_stops', 'qualifying_position']]

# Drop rows with missing data
model_data = model_data.dropna(subset=['positionOrder', 'avg_lap_time_ms', 'num_pit_stops', 'qualifying_position'])

df = results_merged.dropna(subset=[
    'positionOrder', 'avg_lap_time_ms', 'num_pit_stops', 'qualifying_position'
])

# COMMAND ----------

display(df)

# COMMAND ----------

In [0]:
#Cleaning up the data
# Step 1: Replace '\\N' with NaN in problematic columns BEFORE split
cols_with_N = [
    'position', 'time', 'milliseconds', 'fastestLap', 'fastestLapTime',
    'fastestLapSpeed', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time',
    'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date',
    'sprint_time', 'number_driver'
]
cols_existing = [col for col in cols_with_N if col in df.columns]
df[cols_existing] = df[cols_existing].replace('\\N', np.nan)

# Step 2: Convert numeric-like columns
numeric_like = ['position', 'milliseconds', 'fastestLapSpeed', 'number_driver']
numeric_existing = [col for col in numeric_like if col in df.columns]
for col in numeric_existing:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 3: Drop non-numeric columns OR encode them if needed
df = df.select_dtypes(include=[np.number])  # Simple: drop all non-numeric cols

# Step 4: Drop remaining rows with missing data or use imputer
df = df.dropna()

# Step 5: Define features and target
X = df.drop(columns=["positionOrder"])
y = df["positionOrder"]

# MAGIC %md
# MAGIC Perform a train/test split.

# COMMAND ----------
# Step 6: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Step 7: Train model and log with MLflow
with mlflow.start_run(run_name="Basic RF Experiment") as run:
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")

    # Metrics
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)
    print(f"  mse: {mse}")
  
    # Log metrics
    mlflow.log_metric("mse", mse)
  
    runID = run.info.run_uuid
    experimentID = run.info.experiment_id
  
    print("Inside MLflow Run with run_id {} and experiment_id {}".format(runID, experimentID))
 

In [0]:
# COMMAND ----------

# MAGIC %md-sandbox
# MAGIC ### Parameters, Metrics, and Artifacts
# MAGIC 
# MAGIC But wait, there's more!  In the last example, you logged the run name, an evaluation metric, and your model itself as an artifact.  Now let's log parameters, multiple metrics, and other artifacts including the feature importances.
# MAGIC 
# MAGIC First, create a function to perform this.
# MAGIC 
# MAGIC <img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> To log artifacts, we have to save them somewhere before MLflow can log them.  This code accomplishes that by using a temporary file that it then deletes.

# COMMAND ----------

def log_rf(experimentID, run_name, params, X_train, X_test, y_train, y_test):
  import os
  import matplotlib.pyplot as plt
  import mlflow.sklearn
  import seaborn as sns
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
  import tempfile

  with mlflow.start_run(experiment_id=experimentID, run_name=run_name) as run:
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")

    # Log params
    [mlflow.log_param(param, value) for param, value in params.items()]

    # Create metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print("  mse: {}".format(mse))
    print("  mae: {}".format(mae))
    print("  R2: {}".format(r2))

    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)  
    mlflow.log_metric("r2", r2)  
    
    # Create feature importance
    importance = pd.DataFrame(list(zip(df.columns, rf.feature_importances_)), 
                                columns=["Feature", "Importance"]
                              ).sort_values("Importance", ascending=False)
    
    # Log importances using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="feature-importance-", suffix=".csv")
    temp_name = temp.name
    try:
      importance.to_csv(temp_name, index=False)
      mlflow.log_artifact(temp_name, "feature-importance.csv")
    finally:
      temp.close() # Delete the temp file
    
    # Create plot
    fig, ax = plt.subplots()

    sns.residplot(x=predictions, y=y_test, lowess=True, ax=ax)
    plt.xlabel("Predicted values for Price ($)")
    plt.ylabel("Residual")
    plt.title("Residual Plot")

    # Log residuals using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="residuals-", suffix=".png")
    temp_name = temp.name
    try:
      fig.savefig(temp_name)
      mlflow.log_artifact(temp_name, "residuals.png")
    finally:
      temp.close() # Delete the temp file
      
    display(fig)
    return run.info.run_uuid

# COMMAND ----------

# MAGIC %md
# MAGIC Run with new parameters.

# COMMAND ----------

params1 = {
  "n_estimators": 100,
  "max_depth": 5,
  "random_state": 42
}

log_rf(experimentID, "Run 1: 100 Estimators, Depth 5", params1, X_train, X_test, y_train, y_test)

# COMMAND ----------

params2 = {
  "n_estimators": 200,
  "max_depth": 5,
  "random_state": 42
}

log_rf(experimentID, "Run 2: 200 Estimators, Depth 5", params2, X_train, X_test, y_train, y_test)

# COMMAND ----------

params3 = {
  "n_estimators": 300,
  "max_depth": 5,
  "random_state": 42
}

log_rf(experimentID, "Run 3: 300 Estimators, Depth 5", params3, X_train, X_test, y_train, y_test)

# COMMAND ----------

params4 = {
  "n_estimators": 100,
  "max_depth": 10,
  "random_state": 42
}

log_rf(experimentID, "Run 4: 100 Estimators, Depth 10", params4, X_train, X_test, y_train, y_test)

# COMMAND ----------

params5 = {
  "n_estimators": 200,
  "max_depth": 10,
  "random_state": 42
}

log_rf(experimentID, "Run 5: 200 Estimators, Depth 10", params5, X_train, X_test, y_train, y_test)

# COMMAND ----------

params6 = {
  "n_estimators": 300,
  "max_depth": 10,
  "random_state": 42
}

log_rf(experimentID, "Run 6: 300 Estimators, Depth 10", params6, X_train, X_test, y_train, y_test)

# COMMAND ----------

params7 = {
  "n_estimators": 100,
  "max_depth": 15,
  "random_state": 42
}

log_rf(experimentID, "Run 7: 100 Estimators, Depth 15", params7, X_train, X_test, y_train, y_test)

# COMMAND ----------

params8 = {
  "n_estimators": 200,
  "max_depth": 15,
  "random_state": 42
}

log_rf(experimentID, "Run 8: 200 Estimators, Depth 15", params8, X_train, X_test, y_train, y_test)

# COMMAND ----------

params9 = {
  "n_estimators": 300,
  "max_depth": 15,
  "random_state": 42
}

log_rf(experimentID, "Run 9: 300 Estimators, Depth 15", params9, X_train, X_test, y_train, y_test)

# COMMAND ----------

params10 = {
  "n_estimators": 500,
  "max_depth": 20,
  "random_state": 42
}

log_rf(experimentID, "Run 10: 500 Estimators, Depth 20", params10, X_train, X_test, y_train, y_test)

# COMMAND ----------


The best model for this is **run 3**.
- R² = 0.99999969, shows the highest variance explained by all the runs.
- MSE = 0.000004887, the lowest mean squared error out of all the runs.
- MAE = 0.0001450, also the lowest mean absolute error out of all the runs.

This suggests that the model trained with n_estimators=300 and max_depth=5 provided the most accurate predictions with the best generalization performance. Therefore, Run 3 is the best model.